# Magnetic field in dust dominated bouncing cosmology

In this notebook we develop the necessary objects to compute the power spectrum (and other observables) for magnetic field production in the contraction phase of a bouncing cosmology dominated by a dust-like fluid.

The scale factor for this model is given by:
\begin{equation}
\frac{a(\eta)}{a_0} = \frac{1}{x_b} \left[\eta^2\gamma+\sqrt{1+\left(\frac{\eta}{\eta_b}\right)^2}\right], \quad x \equiv \frac{a_0}{a}, \quad x_b \equiv \frac{a_0}{a_b},
\end{equation}
where $a_b$ gives the value of the scale factor at the bounce ($\eta = 0$), and $\eta_b$ the bounce time-scale. We can write the constants $\gamma$ and $\eta_b$ in terms of the usual densities $\Omega_r$ and $\Omega_m$, computing the Hubble function $H = a^\prime/a^2$ away from the bounce phase $\eta\gg\eta_b$ (${}^\prime$ represents derivatives with respect to the conformal time $\eta$), i.e.,
\begin{equation}
H^2=\left(\frac{a^\prime}{a^2}\right)^2 = \frac{1}{a_0^2}\left(\frac{4\gamma x^3}{x_b}+\frac{x^4}{x_b^2\eta_b^2}\right) = R_H^2\left(\Omega_m x^3 + \Omega_r x^4\right) \quad \Rightarrow \quad \gamma = \frac{a_0^2x_b\Omega_m}{4R_H^2}, \quad \eta_b = \frac{R_H}{a_0x_b\sqrt{\Omega_r}},
\end{equation}
where $R_H \equiv 1/H_0$ is the Hubble radius today and $\Omega_m$ and $\Omega_r$ are the dimensionless densities today.

One can see that the problem can be parametrized by $\Omega_m$, $\Omega_r$ and $x_b$ by writing all quantities in units of the conformal time today:
\begin{equation}
\bar\eta \equiv \frac{a_0}{R_H} \eta, \quad \bar\gamma \equiv \frac{R_H^2}{a_0^2}\gamma =  \frac{x_b\Omega_m}{4}, \quad \bar\eta_b \equiv \frac{a_0}{R_H} \eta_b = \frac{1}{x_b\sqrt{\Omega_r}}.
\end{equation}
Now, the scale factor is given by 
\begin{equation}
\frac{a(\bar\eta)}{a_0} = \frac{1}{x_b} \left[\bar\eta^2\bar\gamma+\sqrt{1+\left(\frac{\bar\eta}{\bar\eta_b}\right)^2}\right].
\end{equation}
From here on, we are dropping the over-bars from all quantities and always assume the quantities are written in unit of conformal time today.


## Loading NumCosmo

The first step is to load both NumCosmo and NumCosmoMath libraries.

In [3]:
try:
  import gi
  gi.require_version('NumCosmo', '1.0')
  gi.require_version('NumCosmoMath', '1.0')
except:
  pass

import math
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm


# Initializing the library

In [6]:
Ncm.cfg_init ()

# Complex structure quantization object

Below we define our object as a child of the complex structure quantization object Ncm.CSQ1D

In [7]:
class PyCSQ1DMagDust (Ncm.CSQ1D):
  def __init__ (self, Omega_m = 0.3, xb = 1.0e25, cc = 1.0):
    Ncm.CSQ1D.__init__ (self)

    self.Omega_m = Omega_m
    self.xb      = xb
    self.alpha   = 1.5 * math.sqrt (Omega_m * xb**3)
    self.cc      = cc

  def y_t (self, t):
    return np.cbrt (1.0 + (self.alpha * t)**2)

  def H_t (self, t):    
    return 2.0 / 3.0 * self.alpha**2 * t / (1.0 + (self.alpha * t)**2)

  def F_t (self, t):
    alphat  = self.alpha * t
    alphat2 = alphat * alphat
    return 0.25 * (1.0 + self.cc**2 * (alphat2 + 3.0) / (alphat2 + 1.0)**2)
    
  def do_eval_xi (self, model, t, k):
    return math.log (k) + math.log (self.F_t (t))

  def do_eval_nu (self, model, t, k):
    return k / self.y_t (t)

  def do_eval_F1 (self, model, t, k):
    alphat  = self.alpha * t
    alphat2 = alphat * alphat
    
    T1 = -2.0 * self.cc**2 * self.alpha**2 * t * (alphat2 + 5.0)
    T2 = alphat2 + 1.0
    T3 = T2**2
    T4 = self.cc**2 * (alphat2 + 3.0)

    return (T1 / (T2 * (T3 + T4))) / (2.0 * self.do_eval_nu (model, t, k))

  def do_eval_F2 (self, model, t, k):
    cc      = self.cc
    cc2     = cc**2
    alpha2  = self.alpha**2
    alphat  = self.alpha * t
    alphat2 = alphat * alphat
    alphat4 = alphat2 * alphat2
    alphat6 = alphat4 * alphat2
    alphat8 = alphat4 * alphat4

    T1  = (2.0 * cc2 * alpha2 * (-15.0 * (1.0 + 3.0 * cc2) + (26.0 + 3.0 * cc2) * alphat2 + (104.0 + 17.0 * cc2) * alphat4 + (70.0 + cc2) * alphat6 + 7.0 * alphat8)) / (3.0 * ((1.0 + alphat2)**3 + cc2 * (3.0 + 4.0 * alphat2 + alphat4))**2)
    Fpp = (cc2 * alpha2 * (-5.0 + 22.0 * alphat2 + 3.0 * alphat4)) / (2.0 * (1.0 + alphat4)**4)
    
    return T1 / (2.0 * self.do_eval_nu (model, t, k))**2

  def do_prepare (self, model):
    pass 
